# Первичный анализ данных с Pandas

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/adult.data.csv')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


1. Сколько мужчин и женщин (признак sex) представлено в этом наборе данных?

In [3]:
data['sex'].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

2. Каков средний возраст (признак age) женщин?

In [5]:
data[data['sex'] == 'Female']['age'].mean()

36.85823043357163

3. Какова доля граждан Германии (признак native-country)?

In [6]:
data['native-country'].value_counts(normalize=True)['Germany']

0.004207487485028101

4-5. Каковы средние значения и среднеквадратичные отклонения возраста тех, 
кто получает более 50K в год (признак salary) и тех, кто получает менее 50K в год?

In [7]:
data.groupby(['salary'])['age'].agg([np.mean, np.std])

,mean,std
salary,,
<=50K,36.783738,14.020088
>50K,44.249841,10.519028


6. Правда ли, что люди, которые получают больше 50k, имеют как минимум высшее образование? 
(признак education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters или Doctorate)

In [8]:
pd.crosstab(data['education'], data['salary'])

salary,<=50K,>50K
education,,
10th,871,62
11th,1115,60
12th,400,33
1st-4th,162,6
5th-6th,317,16
7th-8th,606,40
9th,487,27
Assoc-acdm,802,265
Assoc-voc,1021,361


In [16]:
data[(data['salary'] == '>50K') & ~((data['education'] == 'Bachelors')|
                                   (data['education'] == 'Prof-school')|
                                   (data['education'] == 'Assoc-acdm')|
                                   (data['education'] == 'Assoc-voc')|
                                   (data['education'] == 'Masters')|(data['education'] == 'Doctorate'))].shape[0]

3306

In [ ]:
3306 человек не имеют высшее образование - не правда

7. Выведите статистику возраста для каждой расы (признак race) и каждого пола. 
Используйте groupby и describe. Найдите таким образом максимальный возраст мужчин расы Amer-Indian-Eskimo.

In [9]:
data[data['sex'] == 'Female'].groupby(['race'])['age'].describe(percentiles=[])

,count,mean,std,min,50%,max
race,,,,,,
Amer-Indian-Eskimo,119.0,37.117647,13.114991,17.0,36.0,80.0
Asian-Pac-Islander,346.0,35.089595,12.300845,17.0,33.0,75.0
Black,1555.0,37.854019,12.637197,17.0,37.0,90.0
Other,109.0,31.678899,11.631599,17.0,29.0,74.0
White,8642.0,36.811618,14.329093,17.0,35.0,90.0


In [10]:
data[data['sex'] == 'Male'].groupby(['race'])['age'].describe(percentiles=[])

,count,mean,std,min,50%,max
race,,,,,,
Amer-Indian-Eskimo,192.0,37.208333,12.049563,17.0,35.0,82.0
Asian-Pac-Islander,693.0,39.073593,12.883944,18.0,37.0,90.0
Black,1569.0,37.682600,12.882612,17.0,36.0,90.0
Other,162.0,34.654321,11.355531,17.0,32.0,77.0
White,19174.0,39.652498,13.436029,17.0,38.0,90.0


8. Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак marital-status)? 
Женатыми считаем тех, у кого marital-status начинается с Married (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.

In [11]:
df = data[data['sex'] == 'Male']
pd.crosstab(df['marital-status'], df['salary']) 

salary,<=50K,>50K
marital-status,,
Divorced,1487,284
Married-AF-spouse,5,4
Married-civ-spouse,7381,5938
Married-spouse-absent,190,23
Never-married,5591,325
Separated,345,49
Widowed,129,39


In [5]:
marriedMen = data[(data['salary'] == '>50K') & (data['sex'] == 'Male') &
                  ((data['marital-status'] == 'Married-civ-spouse')|
                  (data['marital-status'] == 'Married-spouse-absent')|
                  (data['marital-status'] == 'Married-AF-spouse'))].shape[0]

notmarriedMen = data[(data['salary'] == '>50K') & (data['sex'] == 'Male') &~
                  ((data['marital-status'] == 'Married-civ-spouse')|
                  (data['marital-status'] == 'Married-spouse-absent')|
                  (data['marital-status'] == 'Married-AF-spouse'))].shape[0]

print ("Мужчин, которые зарабатывают больше 50К: Женатых: %i. Холостых: %i" % (marriedMen,notmarriedMen))


if marriedMen > notmarriedMen:
    print("Больше женатых.")
else:
    print("Больше холостых.")

Мужчин, которые зарабатывают больше 50К: Женатых: 1747. Холостых: 197
Больше женатых.


9. Какое максимальное число часов человек работает в неделю (признак hours-per-week)? 
Сколько людей работают такое количество часов и каков среди них процент зарабатывающих много?

In [3]:
maxWorkHrs = data['hours-per-week'].max()

workaholicsNum = data[data['hours-per-week'] == 99].shape[0]

richWorkaholicsNum= data[(data['salary'] == '>50K') & (data['hours-per-week'] == 99)].shape[0]

percentRichWorkaholics = (richWorkaholicsNum/workaholicsNum)*100;

print("Максимальное количество рабочих часов: %i, какое количество людей так работает: %i,  процент зарабатывающих много: %.0f%%." % (maxWorkHrs,workaholicsNum,percentRichWorkaholics))

Максимальное количество рабочих часов: 99, какое количество людей так работает: 85,  процент зарабатывающих много: 29%.


10. Посчитайте среднее время работы (hours-per-week) зарабатывающих мало и много (salary) для каждой страны (native-country).

In [20]:
pd.crosstab(data['native-country'], data['salary'], 
           values=data['hours-per-week'], aggfunc=np.mean).T

native-country,?,Cambodia,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
salary,,,,,,,,,,,,,,,,,,,,,
<=50K,40.164760,41.416667,37.914634,37.381818,38.684211,37.985714,42.338235,38.041667,36.030928,40.483333,...,41.939394,38.470588,39.444444,40.15625,33.774194,42.866667,37.058824,38.799127,37.193548,41.6
>50K,45.547945,40.000000,45.641026,38.900000,50.000000,42.440000,47.000000,48.750000,45.000000,44.533333,...,41.500000,39.416667,46.666667,51.43750,46.800000,58.333333,40.000000,45.505369,39.200000,49.5
